# Классификация покемонов **часть 2**

## Как загрузить колллаб на свой гугл диск

Чтобы иметь возможность запустить блоки кода в Google Colab, вам необходимо сохранить его на своем Google Disk.
Нажмите на вкладку File в левом верхнем углу, у вас откроется окно с различными пунктами, как видно на картинке ниже. Выберите пункт сохранить копию на диске (Save a copy in Drive).  

![alt text](https://sun2.is74.userapi.com/XPz5mAsxV4dq9DaQummKA-BUZO1zoa6yX5cj8w/Wdnxs_GHtv8.jpg)

У вас должна автоматически открыться новая вкладка с сохраненной копией Google Colab.

![alt text](https://sun1.is74.userapi.com/qheclDwr9RphL3yXKsGjyEbrYeYbCzp_cpM9cg/bMUYV8loGc4.jpg)

Нажмите на вкладку Edit и выберите пункт Notebook settings. 

![alt text](https://sun1.is74.userapi.com/vSOJHyGKJFg0SO_F6z9y3lwRSN5-e8Uc7CC9xQ/E-Q6EeGtmNQ.jpg)

Убедитесь, что у вас выбрано устройсво GPU.

![alt text](https://sun1.is74.userapi.com/5tLEM9N4hWVdbBNbBG88PaT8ByPR6va9hVpfdw/XcvHl6aKZs4.jpg)

---
## Что будем делать

На этом занятии мы реализуем и обучим сверточную нейронную сеть, которая будет способна распознавать пять разных видов покемонов: Псайдак, Сквиртл, Чармандер, Пикачу и Бульбазавр.

![alt text](https://sun1.is74.userapi.com/c0xGQDqIsYMEyhLDCoG8TCyhUWQoQDCAHXaydg/9O_kL_yJRqY.jpg)

Мы рассмотрим подход с переносом обучения (transfer learning). Суть данного подхода состоит в том, что нейронные сети, которые были предварительно обучены на большом наборе данных, можно использовать для решения задач в других областях. Данный подход не только позволяет повысить точность за счет того, что изначально сеть была обучена на большом количестве данных, но также уменьшить временные и ресурсные затраты на обучение.

## Импорт библиотек
Загрузим необходимые бибилиотеки для реализации сверточной нейронной сети.

Numpy - это библиотека, добавляющая поддержку больших многомерных массивов и матриц, вместе с большой библиотекой высокоуровневых математических функций для операций с этими массивами.

Keras - открытая нейросетевая библиотека для оперативной работы с сетями глубокого обучения.

Matplotlib - библиотека для визуализации данных.

In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential  
from keras.layers import  Dense
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from matplotlib import pyplot as plt
import scipy
from scipy import misc,ndimage
import tensorflow as tf
%matplotlib inline
from sklearn.datasets import load_files
from keras.utils import np_utils
from glob import glob 
from keras.utils.np_utils import to_categorical   
import math  
import cv2 
import os
from zipfile import ZipFile

## Загрузка данных
Выполним загрузку набора изображения, на чем будет дообучаться наша нейронная сеть.

Выберите вкладку Files слева в вертикальном меню.

![alt text](https://sun1.is74.userapi.com/kFOVcPGnc82bkA5zoKykCt3j2q6xMa9SfmSTMg/zs5A0wjhmRs.jpg)

Нажмите на кнопку Upload, чтобы загрузить датасет **Pokemon_dataset_big.zip** в Google Colab. [Ссылка на датасеты](https://drive.google.com/drive/folders/18vQ6tXnDm21mqn-02U-40jUvUaL1gYo9?usp=sharing)


![alt text](https://sun2.is74.userapi.com/fDp8RuYK5Mp-ULMxV0WtUfUub4HH4QBDsu_vSg/A16VSk2vl5s.jpg)

Поместить архив необходимо в текущую директорую ( РЯДОМ с папкой sample_data) 

In [ ]:
# !!не забудьте указать название датасета, который будете загружать
fileName = 'Pokemon_dataset_big.zip'
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

## Препроцессинг
Зафиксируем пути для тренировачного, валидационного и тестового набора.

In [ ]:
# !!обновите тренировачные, валидационные и тестовые пути
train_path='Pokemon_dataset_big/train'
valid_path='Pokemon_dataset_big/valid'
test_path='Pokemon_dataset_big/test'
  
img_width, img_height = 224, 224  
batch_size = 8

Выполним простую аугментацию изображений из тренировочного набора. 

In [ ]:
datagen=ImageDataGenerator(rescale=1./255)
train_datagen=ImageDataGenerator(rotation_range=90,
                                rescale=1./255,
                                horizontal_flip=True)

## Загрузка InceptionV3
Выполним загрузку предварительно обученной нейронной сети InceptionV3, без последнего слоя. Таким образом, будет использоваться только сверточная часть нейронной сети. Она будет извлекать признаки из изображений, а за их классификацию будет отвечать наша нейронная сеть.

![alt text](https://sun2.is74.userapi.com/J07QPdqMuK1MKCbO60vjP-vcyzW66d6Urd0gmw/jI9dhkNQOfE.jpg)



In [ ]:
from keras import applications
base_model = applications.InceptionV3(include_top=False, weights='imagenet')

## Извлечение карт признаков из InceptionV3
В следующих двух блоках загрузим тренировочный и валидационный наборы в предварительно обученную нейронную сеть InceptionV3 и извлечем карты признаков (bootleneck features), которые в дальнейшем будем использовать в нашем классификаторе для распознавания покемонов.

In [ ]:
generator = train_datagen.flow_from_directory(  
    train_path,  
    target_size=(img_width, img_height),  
    batch_size=batch_size,  
    class_mode=None,  
    shuffle=False)  

nb_train_samples = len(generator.filenames)  
num_classes = len(generator.class_indices)  
print(num_classes)
   
predict_size_train = int(math.ceil(nb_train_samples/batch_size))  
   
bottleneck_features_train = base_model.predict_generator( generator, 
    predict_size_train, verbose=1)  
   
np.save('bottleneck_features_train_pokemon.npy', bottleneck_features_train)  

generator = datagen.flow_from_directory(  
     valid_path,  
     target_size=(img_width, img_height),  
     batch_size=batch_size,  
     class_mode=None,  
     shuffle=False)  
   
nb_validation_samples = len(generator.filenames)  
   
predict_size_validation = int(math.ceil(nb_validation_samples/batch_size))  
   
bottleneck_features_validation = base_model.predict_generator(  
     generator, predict_size_validation,verbose=1)  
   
np.save('bottleneck_features_validation_pokemon.npy', bottleneck_features_validation)  


In [ ]:
generator_top = train_datagen.flow_from_directory(  
         train_path,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode='categorical',  
         shuffle=False)  
   
nb_train_samples = len(generator_top.filenames)  
num_classes = len(generator_top.class_indices)  
   
  
train_data = np.load('bottleneck_features_train_pokemon.npy')  
   
train_labels = generator_top.classes  
   
train_labels = to_categorical(train_labels, num_classes=num_classes) 

generator_top = datagen.flow_from_directory(  
         valid_path,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode='categorical',  
         shuffle=False)  
   
nb_validation_samples = len(generator_top.filenames)  
   
validation_data = np.load('bottleneck_features_validation_pokemon.npy')  
   
validation_labels = generator_top.classes  
validation_labels = to_categorical(validation_labels, num_classes=num_classes) 

## Реализация классификатора
Реализуем модель нейронной сети, используя библиотеку Keras. На вход классификатора подадим признаки, полученные ранее из сверточной части нейронной сети InceptionV3.

In [ ]:
res_model = Sequential()
res_model.add(GlobalAveragePooling2D(input_shape=train_data.shape[1:]))
# !!не забудьте изменить количество нейронов в выходном слое
res_model.add(Dense(5, activation='softmax'))

res_model.summary()

## Компиляция и обучение нейронной сети
Скомпилируем нашу модель и запустим обучение.

In [ ]:
res_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
res_model.fit(train_data,train_labels,
                    validation_data=(validation_data,validation_labels),
                    epochs=30,batch_size=batch_size,verbose=1)

## Классы покемонов
Создадим массив из имен покемонов.

In [ ]:
pokemon_names = [item[26:-1] for item in sorted(glob(train_path+'/*/'))]
pokemon_names

## Предобработка тестовых изображений
Запустим необходимые функции, которые понадобяться для предобработки и загрузки тестового изображения.

In [ ]:
from keras.preprocessing import image 
def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def extract_InceptionV3(tensor):
	from keras.applications.inception_v3 import InceptionV3, preprocess_input
	return InceptionV3(weights='imagenet', include_top=False).predict(preprocess_input(tensor))
 
def vgg_predict_pokemon(img_path):
    bottleneck_feature = extract_InceptionV3(path_to_tensor(img_path))
    predicted_vector = res_model.predict(bottleneck_feature)
    name = pokemon_names[np.argmax(predicted_vector)]
    img = cv2.imread(img_path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgplot = plt.imshow(cv_rgb)
    plt.axis('off')
    return format(name)

## Распознавание тестового изображения

### *ЗАДАНИЕ*
Выполните загрузку любого тестового изображения из папки test c локального компьютера. Не забудьте импортировать необходимый пакет из google colab.

Вам нужно написать код по загрузке изображения в ячейке ниже.

In [ ]:
# напишите код по загрузке изображения после комментария



In [ ]:
vgg_predict_pokemon(next(iter(file_name)))

## Задания для второй практики

### Загрузка изображения

После выполнения задания, вам нужно будет прикрепить измененный Google Colab в курс на платформе. Для того чтобы скачать файл нажмите выберите File -> Download .ipynb.
![alt text](https://sun2.is74.userapi.com/EqGA1cIb3fITkfwnoV0nLgYgEM4b0xzvz94t2A/_ksVBmhhzWU.jpg)



### Задание на обучение нейронной сети на другом наборе данных
Добавим еще один класс покемона в датасет. Необходимо будет обучить сверточную нейронную сеть на новом наборе данных, который будет включать в себя 6 видов покемонов.

[ссылка на датасет](https://drive.google.com/drive/folders/1-T0IFPu9_cAfHNVRzrgH6o4Xpw_EIBbf?usp=sharing)

Вам необходимо будет проделать те же шаги по загрузке датасета, что и в начале и изменить код в некоторых местах.

Блоки кода, в которых надо что-то поменять отмечены комментариями 

```
#!!какой-то текст
```

После обучения и тестирования нейронной сети, вам необходимо будет прикрепить измененный Google Colab в курс на платформе. Для того чтобы скачать файл нажмите выберите File -> Download .ipynb.
